# ခရီးသွားစရိတ်တောင်းခံမှု ချက်ချင်းသုံးသပ်ခြင်း

ဒီ notebook က plugin တွေကို အသုံးပြုပြီး ဒေသခံလက်ခံလက်မှတ်ပုံတွေကနေ ခရီးသွားစရိတ်တွေကို အဆင့်ဆင့်လုပ်ဆောင်၊ စရိတ်တောင်းခံအီးမေးလ်ကို ဖန်တီးပြီး စရိတ်ဒေတာကို pie chart နဲ့ မြင်သာအောင် ဖော်ပြပေးတဲ့ agent တွေကို ဖန်တီးပုံကို ပြသထားပါတယ်။ Agent တွေက အလုပ်အကိုင်အခြေအနေအပေါ် မူတည်ပြီး လိုအပ်တဲ့ function တွေကို အလိုအလျောက် ရွေးချယ်ပါတယ်။

အဆင့်များ:
1. OCR Agent က ဒေသခံလက်ခံလက်မှတ်ပုံကို အဆင့်ဆင့်လုပ်ဆောင်ပြီး ခရီးသွားစရိတ်ဒေတာကို ထုတ်ယူပေးသည်။
2. Email Agent က စရိတ်တောင်းခံအီးမေးလ်ကို ဖန်တီးပေးသည်။

### ခရီးသွားစရိတ်အခြေအနေတစ်ခု၏ ဥပမာ:
သင်ဟာ အခြားမြို့တစ်မြို့ကို စီးပွားရေးအစည်းအဝေးတစ်ခုအတွက် ခရီးသွားနေတဲ့ ဝန်ထမ်းတစ်ဦးဖြစ်တယ်လို့ စဉ်းစားပါ။ သင့်ကုမ္ပဏီမှာ ခရီးသွားစရိတ်ဆိုင်ရာ သင့်တော်တဲ့ စရိတ်အားလုံးကို ပြန်လည်ပေးဆောင်ပေးမယ်ဆိုတဲ့ မူဝါဒရှိပါတယ်။ ခရီးသွားစရိတ်အမျိုးအစားတွေကို အောက်ပါအတိုင်း ခွဲခြားဖော်ပြထားပါတယ်။

- သယ်ယူပို့ဆောင်ရေး:
သင့်နေအိမ်မြို့ကနေ အစည်းအဝေးမြို့ကို သွားလာခရီးစဉ်အတွက် လေယာဉ်လက်မှတ်။
လေဆိပ်ကို သွားလာဖို့ Taxi သို့မဟုတ် ride-hailing ဝန်ဆောင်မှု။
အစည်းအဝေးမြို့မှာ ဒေသခံသယ်ယူပို့ဆောင်ရေး (အများပြည်သူသယ်ယူပို့ဆောင်ရေး၊ ကားငှားခြင်း၊ Taxi စသည်တို့)။

- နေထိုင်ခန်း:
အစည်းအဝေးနေရာအနီးမှာရှိတဲ့ အလယ်အလတ်စီးပွားရေးဟိုတယ်မှာ သုံးညအိပ်စက်ခြင်း။

- အစားအသောက်:
ကုမ္ပဏီရဲ့ per diem မူဝါဒအရ နေ့စဉ်မနက်စာ၊ နေ့လယ်စာ၊ ညစာအတွက် စရိတ်။

- အခြားစရိတ်များ:
လေဆိပ်မှာ ကားရပ်နားခ။
ဟိုတယ်မှာ အင်တာနက်အသုံးပြုခ။
Tips သို့မဟုတ် ဝန်ဆောင်မှုအခကြေးငွေသေးသေးလေးများ။

- အထောက်အထား:
လေယာဉ်လက်မှတ်၊ Taxi, ဟိုတယ်, အစားအသောက် စသည်တို့အတွက် လက်ခံလက်မှတ်များနှင့် ပြန်လည်ပေးဆောင်မှုအတွက် ပြည့်စုံတဲ့ စရိတ်တောင်းခံအစီရင်ခံစာကို တင်ပြပါ။


## လိုအပ်သောစာကြည့်တိုက်များကို သွင်းယူပါ

Notebook အတွက် လိုအပ်သော စာကြည့်တိုက်များနှင့် module များကို သွင်းယူပါ။


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## အသုံးစရိတ် မော်ဒယ်များ သတ်မှတ်ခြင်း

တစ်ခုချင်းအသုံးစရိတ်များအတွက် Pydantic မော်ဒယ်တစ်ခုဖန်တီးပြီး အသုံးပြုသူ၏မေးခွန်းကို ဖွဲ့စည်းထားသော အသုံးစရိတ်ဒေတာအဖြစ် ပြောင်းလဲပေးရန် ExpenseFormatter class တစ်ခု ဖန်တီးပါ။

တစ်ခုချင်းအသုံးစရိတ်ကို အောက်ပါပုံစံအတိုင်း ကိုယ်စားပြုပါမည်။
`{'date': '07-Mar-2025', 'description': 'flight to destination', 'amount': 675.99, 'category': 'Transportation'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## အေးဂျင့်များကို သတ်မှတ်ခြင်း - အီးမေးလ်ကို ဖန်တီးခြင်း

ကုန်ကျစရိတ်တောင်းခံမှုအတွက် အီးမေးလ်တစ်စောင် ဖန်တီးရန် အေးဂျင့် class တစ်ခုကို ဖန်တီးပါ။
- ဒီအေးဂျင့်က `kernel_function` အလှဆင်ကို အသုံးပြုပြီး ကုန်ကျစရိတ်တောင်းခံမှုအတွက် အီးမေးလ်တစ်စောင် ဖန်တီးတဲ့ function ကို သတ်မှတ်ပါတယ်။
- ကုန်ကျစရိတ်စုစုပေါင်းကိုတွက်ချက်ပြီး အီးမေးလ်အကြောင်းအရာထဲမှာ အသေးစိတ်ကို ဖော်ပြပေးပါတယ်။


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# ခရီးသွားစရိတ်များကို ဘောင်ချာပုံများမှ ထုတ်ယူရန် Agent

ခရီးသွားစရိတ်များကို ဘောင်ချာပုံများမှ ထုတ်ယူရန် agent class တစ်ခု ဖန်တီးပါ။
- ဒီ agent သည် `kernel_function` အလှဆင်ကို အသုံးပြုပြီး ခရီးသွားစရိတ်များကို ဘောင်ချာပုံများမှ ထုတ်ယူရန် function တစ်ခုကို သတ်မှတ်ပါသည်။
- OCR (Optical Character Recognition) ကို အသုံးပြု၍ ဘောင်ချာပုံကို စာသားအဖြစ် ပြောင်းပြီး ရက်စွဲ၊ ဖော်ပြချက်၊ ငွေပမာဏနှင့် အမျိုးအစားကဲ့သို့သော သက်ဆိုင်သော အချက်အလက်များကို ထုတ်ယူပါ။


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## အကုန်အကျဆောင်ရွက်ခြင်း

အကုန်အကျများကို ဆောင်ရွက်ရန် လိုအပ်သော အေးဂျင့်များကို ဖန်တီးပြီး မှတ်ပုံတင်ကာ အလုပ်လုပ်စေသည့် asynchronous function တစ်ခုကို သတ်မှတ်ပါ။
- ဒီ function သည် ပတ်ဝန်းကျင် variable များကို load လုပ်ခြင်း၊ လိုအပ်သော အေးဂျင့်များကို ဖန်တီးခြင်းနှင့် plugin အဖြစ် မှတ်ပုံတင်ခြင်းအားဖြင့် အကုန်အကျများကို ဆောင်ရွက်သည်။
- အေးဂျင့်နှစ်ဦးနှင့်အတူ group chat တစ်ခုကို ဖန်တီးပြီး အကုန်အကျ data အပေါ်မူတည်၍ email နှင့် pie chart ကို ဖန်တီးရန် prompt message တစ်ခုကို ပို့သည်။
- Chat invocation အတွင်း ဖြစ်ပေါ်လာသော အမှားများကို ကိုင်တွယ်ပြီး အေးဂျင့်များကို သေချာစွာ ရှင်းလင်းပေးသည်။


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## အဓိကလုပ်ဆောင်ချက်

အဓိကလုပ်ဆောင်ချက်ကိုသတ်မှတ်ပြီး console ကိုရှင်းလင်းရန်နှင့် `process_expenses` လုပ်ဆောင်ချက်ကို asynchronous အနေဖြင့် run ဖို့လုပ်ပါ။


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရ အရင်းအမြစ်အဖြစ် ရှုလေ့လာသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ဘာသာပြန်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွဲအချော်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
